In [42]:
from IPython import display

from matplotlib import pyplot as plt
%matplotlib inline

import math, itertools
import torch.nn as nn
from torch.optim import Adam
from torch.autograd import Variable
from torch.nn import functional, BCELoss
from torchvision import transforms, datasets

In [43]:
download('data/lsun/', 'tower', 'train')

http://lsun.cs.princeton.edu/htbin/download.cgi?tag=latest&category=tower&set=train


## Load Data

In [44]:
def cifar_data():
    compose = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((.5, .5, .5), (.5, .5, .5))
        ])
    return datasets.CIFAR10('./data/cifar10', train=True, transform=compose, download=True)

In [45]:
data = cifar_data()
batch_size = 100
data_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True)

Files already downloaded and verified


## Networks

In [48]:
class GenerativeNet(torch.nn.Module):
    """
    A three hidden-layer generative neural network
    """
    def __init__(self, n_features, n_hidden0, n_hidden1, n_hidden2 , n_out):
        super(GenerativeNet, self).__init__()
        self.linear = torch.nn.Linear(100, 512)
        self.conv1 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=1024, out_channels=512, kernel_size=1, stride=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2)
        )
        self.conv2 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=1, stride=2),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2)
        )
        self.conv3 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=1, ),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2)
        )
        
        self.conv4 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=128, out_channels=3, kernel_size=1),
            nn.BatchNorm2d(),
            nn.LeakyReLU(0.2)
        )
        self.out = torch.nn.Tanh()

    def forward(self, x):
        x = self.linear(x)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        return self.out(x)
    
# Noise
def noise(size):
    return Variable(torch.randn(size, 100))

In [6]:
discriminator = DiscriminativeNet(784, 1024, 512, 256)
generator = GenerativeNet(100, 256, 512, 1024, 784)

## Optimization

## Training